In [1]:
import pandas as pd

df = pd.read_csv("N:/Classes/2022_2FALL/Analytics Day/Data/RollerCoasterDataRATINGS.csv")

In [28]:
df["coaster_park"] = df["Coaster"] + " at " + df["Park"]

df["city_state"] = df["City"] + ", " + df["State"]

df.iloc[0]

Coaster                                              Superman The Escape
Park                                            Six Flags Magic Mountain
Rating                                                               4.0
Top_10_Wooden                                                          0
Top_10_Steel                                                           0
Top_10_Biggest_Drop                                                    0
Top_10_Longest_Length                                                  0
Top_10_Fastest                                                         1
Top_10_Inversions                                                    0.0
Top_10_Duration                                                        0
Total_Num_Top_10                                                       1
Any_Top_10                                                             1
City                                                            Valencia
State                                              

In [47]:
import wikipedia

temp = []
for keyword in df["city_state"].drop_duplicates().tolist():
    try:
        keyword = keyword.replace(";", ":")
        wiki_suggestion = ""
        search_results = wikipedia.search(keyword, results = 1)

        if search_results == []:
            wiki_suggestion = wikipedia.suggest(keyword)
            search_results = wikipedia.search(wiki_suggestion, results = 1)

        wiki_page = wikipedia.page(search_results)

        try:
            lat = float(wiki_page.coordinates[0])
            lon = float(wiki_page.coordinates[1])
        except:
            lat = None
            lon = None

        temp.append([keyword, wiki_page.title, wiki_page.url, lat, lon])
    except:
        temp.append([keyword, None, None, None, None])

c:\ProgramFiles\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\ProgramFiles\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [50]:
temp_df = pd.DataFrame(temp, columns = ["city_state", "title", "url", "latitude", "longitude"])

print("NUMBER OF CITIES LAT/LONG PULLED:", temp_df[temp_df["latitude"].isna() == False].drop_duplicates().shape[0])
print("NUMBER OF CITIES THAT PULL FAILED:", temp_df[temp_df["latitude"].isna() == True].drop_duplicates().shape[0])

NUMBER OF CITIES LAT/LONG PULLED: 62
NUMBER OF CITIES THAT PULL FAILED: 8


In [51]:
temp_df[temp_df["latitude"].isna() == True].drop_duplicates()

,city_state,title,url,latitude,longitude
4,"San Diego, California",None,None,NaN,NaN
7,"Wisconsin Dells, Wisconsin",None,None,NaN,NaN
8,"Louisville, Kentucky","Louisville, Kentucky","https://en.wikipedia.org/wiki/Louisville,_Kent...",NaN,NaN
20,"Branson, Missouri",None,None,NaN,NaN
26,"Santa Claus, Indiana",None,None,NaN,NaN
29,"San Antonio, Texas",San Antonio,https://en.wikipedia.org/wiki/San_Antonio,NaN,NaN
40,"Branson , Missouri",None,None,NaN,NaN
46,"Lake George, New York",None,None,NaN,NaN


In [52]:
df_w_latlon = pd.merge(left = df.reset_index().drop("index", axis = 1),
                       right = temp_df[["city_state", "latitude", "longitude"]].reset_index().drop("index", axis = 1),
                       how = "left",
                       left_on = "city_state",
                       right_on = "city_state"
                      )

In [57]:
df_w_latlon.drop(["coaster_park", "city_state"], axis = 1).to_csv("city_state_latlon.csv")